Import der benötigten Module. Das Modul "ecgdetectors" wird zuerst heruntergeladen und installiert.

In [2]:
from google.colab import files
import csv
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

nb_classes = 2

Upload-Dialog, um die EKG-Daten in die virtuelle Maschine zu laden und zu entpacken (es erfolgt keine Prüfung, ob die richtige Datei hochgeladen wurde)

In [ ]:
uploaded = files.upload()
!unzip training.zip

Alternative: direkter Download aus Google Drive (deutlich schneller)

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/training.zip" .
!unzip training.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  training.zip
replace training/REFERENCE.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training/train_ecg_00001.mat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
def data_preprocessing(ecg_lead,zero_mean_loop):
  if zero_mean_loop ==2:# repeat array to fill 
        if ecg_lead.size<9000: 
          ecg_array_processed = np.array([])
          ecg_array_processed_part2 = np.zeros((9000,), dtype=int)
          #data gets looped till it has size of atleast 9000
          repetitions,remainder=divmod(9000,ecg_lead.size)
          repetitions+=1 
          for x in range(repetitions): 
            ecg_array_processed=np.concatenate((ecg_array_processed,ecg_lead))
          return ecg_array_processed[0:9000],ecg_array_processed_part2[0:9000]      
        if ecg_lead.size>9000:
          #first 9000 can be used unaltered
          ecg_array_processed=ecg_lead[0:9000]
          ecg_array_processed_part2 = np.array([])
          #data above 9000 gets looped till it has size of atleast 9000
          repetitions,remainder=divmod(9000,ecg_lead.size-9000)
          repetitions+=1 
          for x in range(repetitions): 
            ecg_array_processed_part2=np.concatenate((ecg_array_processed_part2,ecg_lead[9000:ecg_lead.size]))
          return ecg_array_processed[0:9000],ecg_array_processed_part2[0:9000]
        if ecg_lead.size==9000:
          ecg_array_processed=ecg_lead[0:9000]
          ecg_array_processed_part2 = np.zeros((9000,), dtype=int)
          return ecg_array_processed,ecg_array_processed_part2[0:9000]
  else: # zero or mean fill of array
        if zero_mean_loop == 0:
          ecg_array_processed = np.zeros((9000,), dtype=int)
          ecg_array_processed_part2 = np.zeros((10000,), dtype=int)
        if zero_mean_loop ==1:
          ecg_mean=np.mean(ecg_lead)   
          ecg_array_processed = np.empty([9000,])
          ecg_array_processed_part2 = np.empty([10000,])
          ecg_array_processed.fill(ecg_mean)
          ecg_array_processed_part2.fill(ecg_mean)
        if ecg_lead.size<9000:      
          ecg_array_processed[0:ecg_lead.size]=ecg_lead[0:ecg_lead.size]
          return ecg_array_processed,ecg_array_processed_part2[0:9000]      
        if ecg_lead.size>9000:
          ecg_array_processed=ecg_lead[0:9000]
          ecg_array_processed_part2[0:ecg_lead.size-9000]=ecg_lead[9000:ecg_lead.size]
          return ecg_array_processed,ecg_array_processed_part2[0:9000]
        if ecg_lead.size==9000:
          ecg_array_processed=ecg_lead[0:9000]
          return ecg_array_processed,ecg_array_processed_part2[0:9000]

In [6]:
sdnn_normal = np.array([])                                # Initialisierung der Feature-Arrays
sdnn_afib = np.array([])
ecg_class = np.array([])
with open('training/REFERENCE.csv') as csv_file:      # Einlesen der Liste mit Dateinamen und Zuordnung
    csv_reader = csv.reader(csv_file, delimiter=',')
    normal_line_count = 0
    afib_line_count = 0
    for row in csv_reader:
      data = sio.loadmat('training/'+row[0]+'.mat')   # Import der EKG-Dateien
      ecg_lead = data['val'][0]
      ecg_array_processed,ecg_array_processed_part2 = data_preprocessing(ecg_lead,2) #data preprocessing 0=fill with zero 1=fill with mean 2=loop
      if row[1] != 'A':                                                   # includes N,O,~
        if normal_line_count != 0:                                        #check if its the first time so that vstack can work properly
          sdnn_normal = np.vstack((sdnn_normal,ecg_array_processed)) 
          if ecg_lead.size>9000:
            sdnn_normal = np.vstack((sdnn_normal,ecg_array_processed_part2))       
        else:
          sdnn_normal=ecg_array_processed
          if ecg_lead.size>9000:
            sdnn_normal = np.vstack((sdnn_normal,ecg_array_processed_part2)) 
        normal_line_count = normal_line_count + 1    
      if row[1]=='A':                                                     # Zuordnung zu "Vorhofflimmern"          
        if afib_line_count != 0:                                          #check if its the first time so that vstack can work properly
          sdnn_afib = np.vstack((sdnn_afib,ecg_array_processed))
          if ecg_lead.size>9000:
            sdnn_afib = np.vstack((sdnn_afib,ecg_array_processed_part2))        
        else:
          sdnn_afib=ecg_array_processed
          if ecg_lead.size>9000:
            sdnn_afib = np.vstack((sdnn_afib,ecg_array_processed_part2))                                 
        afib_line_count = afib_line_count + 1

    if (normal_line_count+ afib_line_count % 100)==0:
        print(str(normal_line_count+ afib_line_count) + "\t Dateien wurden verarbeitet.")

In [7]:
X=np.concatenate((sdnn_normal, sdnn_afib)) #combine Normal and Afib Data

normal_class=np.zeros(sdnn_normal.shape[0])       #combine class labels
normal_class.fill(0)
afib_class=np.zeros(sdnn_afib.shape[0])
afib_class.fill(1)
y=np.concatenate((normal_class, afib_class))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)#split data for testing and training

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [8]:
std_slc = StandardScaler()
std_slc.fit(X_train)
X_train_std = std_slc.transform(X_train)
X_test_std = std_slc.transform(X_test)

In [1]:
# pretrained_base = tf.keras.models.load_model(
#     '../input/cv-course-models/cv-course-models/vgg16-pretrained-base',
# )
# pretrained_base.trainable = False


# model = keras.Sequential([
#     pretrained_base,
#     layers.Flatten(),
#     layers.Dense(6, activation='relu'),
#     layers.Dense(1, activation='sigmoid'),
# ])
#Dense Model
# model = keras.Sequential([
#     layers.Dense(4500, activation='relu', input_shape=[9000]),
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(2200, activation='relu'),  
#     layers.Dropout(0.3),
#     layers.BatchNormalization(), 
#     layers.Dense(1000, activation='relu'),  
#     layers.Dropout(0.3),
#     layers.BatchNormalization(), 
#     layers.Dense(500, activation='relu'),  
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(100, activation='relu'),  
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(10, activation='relu'),  
#     layers.Dropout(0.3),
#     layers.BatchNormalization(),
#     layers.Dense(2, activation='sigmoid'),
# ])

#CNN
# model = keras.Sequential([

#     # First Convolutional Block
#     layers.Conv1D(filters=10, kernel_size=5, activation="relu", padding='same',   #the filter size i chose is random in each layer so far
#                   # give the input dimensions in the first layer
#                   input_shape=[9000,1]),                                        #i am a little confused about input shape and if thats right
#     layers.MaxPool1D(),

#     # Second Convolutional Block
#     layers.Conv1D(filters=10, kernel_size=3, activation="relu", padding='same'),
#     layers.MaxPool1D(),

#     # Third Convolutional Block
#     layers.Conv1D(filters=10, kernel_size=3, activation="relu", padding='same'),
#     layers.MaxPool1D(),

#     # Classifier Head
#     layers.Flatten(),
#     layers.Dense(5, activation='relu', input_shape=[9000]),                   #dense layer sizes could probably be chosen better
#     layers.Dense(10, activation='relu'),    
#     layers.Dense(2, activation='sigmoid'),
# ])

model = keras.Sequential([
    layers.Dense(9000, activation='relu', input_shape=[9000]),
    layers.BatchNormalization(),
    layers.Dense(9000, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(9000, activation='relu'),
])

#Autoencoder
# model = keras.Sequential([
#     #Encoder
#     # First Convolutional Block
#     layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding='same',   #9000,32
#                   input_shape=[9000,1]),                                        
#     layers.MaxPool1D(2,padding='same'),#4500,32
#     layers.BatchNormalization(),  
#     # Second Convolutional Block
#     layers.Conv1D(filters=16, kernel_size=3, activation="relu", padding='same'),#4500,16
#     layers.MaxPool1D(2,padding='same'),#2250,16
#     layers.BatchNormalization(),  

#     # Third Convolutional Block
#     layers.Conv1D(filters=8, kernel_size=3, activation="relu", padding='same'),#2250,8
#     layers.MaxPool1D(2,padding='same'),#1125,8
#     layers.BatchNormalization(),  

#     #Decoder
#     layers.Conv1D(filters=8, kernel_size=3, activation="relu", padding='same'),#1125,8
#     layers.UpSampling1D(2),#2250,8
#     layers.BatchNormalization(),  

#     layers.Conv1D(filters=16, kernel_size=3, activation="relu", padding='same'),#2250,16
#     layers.UpSampling1D(2),#4500,16
#     layers.BatchNormalization(),  

#     layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding='same'),#4500,32
#     layers.UpSampling1D(2),#9000,32
#     layers.BatchNormalization(),  

#     layers.Conv1D(filters=1, kernel_size=3, activation="relu", padding='same'),#9000,1
# ])

#####################Autoencoder#################################
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_percentage_error'],
)

history = model.fit(
    X_train_std, X_train_std,
    validation_data=(X_test_std, X_test_std),
    epochs=30,
    verbose=1,
)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['mean_absolute_percentage_error', 'val_mean_absolute_percentage_error']].plot();

#################For CNN or Dense Model#########################
# model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['binary_accuracy'],
# )
# history = model.fit(                      
#     X_train_std, Y_train,
#     validation_data=(X_test_std, Y_test),
#     epochs=15,
#     verbose=1,
# )
# history_frame = pd.DataFrame(history.history)
# history_frame.loc[:, ['loss', 'val_loss']].plot()
# history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();





NameError: ignored